In [8]:
import numpy as np
from qiskit import ClassicalRegister, QuantumRegister,QuantumCircuit,execute,Aer
from qiskit.tools.visualization import circuit_drawer,plot_histogram


# a =2 

# Set the size circuit
qftsize = 4 #x qubits number
nbits = 4   #y qubits number
#Real SHOR => csize= qftsize 
csize= qftsize + nbits # classical register number

xr = QuantumRegister(qftsize, 'x')
yr = QuantumRegister(nbits,'y')
cr = ClassicalRegister(csize, 'c')

circuit = QuantumCircuit(xr,yr, cr)
# Apply Hadamart gate

for i in range(qftsize):
    circuit.h(xr[i])
circuit.barrier() # separate

# 2 times  controlled circuit
circuit.x(yr[0])
circuit.cx(xr[0],yr[0])
circuit.cx(xr[0],yr[1])

circuit.barrier() # separate


# 4 times mode 15 circuit 2^2 mod 15 = 4
# 4Y mod 15 = 4 y0+ 8y1 +y2 + 2y3.  ( y0 y1 y2 y3) (y1 y0 y3 y2)
# Using Fredkin Gate 

circuit.ccx(xr[1],yr[0],yr[2])
circuit.ccx(xr[1],yr[1],yr[3])
# or
'''
circuit.ccx(xr[0],xr[1],yr[3])
circuit.cx(yr[3],yr[2])
circuit.cx(xr[1],yr[2])


'''
circuit.cx(yr[2],yr[0])
circuit.cx(yr[3],yr[1])

circuit.barrier() #separate

# times a^4 mod N
# 2^4 mod 15 = 1 (no action required)
# Same for a^8

# Measurement

for i in range(qftsize):
    circuit.measure(xr[i], cr[i])
    
# a^x mod N calculation measurement

for i in range(nbits):
    circuit.measure(yr[i],cr[i+qftsize])
    
    
#シミュレーターの指定, select the simulator
simulator = Aer.get_backend('qasm_simulator')
#量子回路のシミュレーターを使った実行, Repeat circuit= #(shots) times
job = execute(circuit, simulator, shots=1000)

#測定結果の取り出し
result = job.result()
counts = result.get_counts(circuit)

#測定結果のヒストグラム出力, Plot the count results on a histogram
plotgraph = plot_histogram(counts)
plotgraph.savefig("measurement_SHORQ2.png",format="png")

#回路図のプロット, Plot the circuit scheme 
diagram = circuit.draw(output='mpl')
diagram.savefig("circuit_SHORQ2.png",format="png")








In [2]:
print("Total count [y3,y2,y1,y0,x3,x2,x1,x0]:", counts )

Total count [y3,y2,y1,y0,x3,x2,x1,x0]: {'00100001': 56, '10000011': 70, '00011000': 59, '01000110': 50, '10000111': 59, '00101101': 68, '10001011': 61, '00101001': 82, '00100101': 60, '00011100': 54, '00010000': 67, '00010100': 56, '10001111': 63, '01001010': 68, '01001110': 68, '01000010': 59}


In [10]:
# Convert to (X,Y)  and verify that Y= 2^X mod 15

scounts = sorted(counts.items(), key= lambda x:x[0][4:8]) # sort xbits

for i in scounts:
    k =i[0]
    v= i[1]
    ybits = k[0:4]  #get the y bits
    xbits = k[4:8]  # get the x bits
    print('(X,Y)= ', xbits,ybits, "=(", int(xbits,2), ',', int(ybits,2),'): count', v )
    
# period of 4




(X,Y)=  0000 0001 =( 0 , 1 ): count 57
(X,Y)=  0001 0010 =( 1 , 2 ): count 66
(X,Y)=  0010 0100 =( 2 , 4 ): count 66
(X,Y)=  0011 1000 =( 3 , 8 ): count 73
(X,Y)=  0100 0001 =( 4 , 1 ): count 52
(X,Y)=  0101 0010 =( 5 , 2 ): count 58
(X,Y)=  0110 0100 =( 6 , 4 ): count 52
(X,Y)=  0111 1000 =( 7 , 8 ): count 63
(X,Y)=  1000 0001 =( 8 , 1 ): count 81
(X,Y)=  1001 0010 =( 9 , 2 ): count 69
(X,Y)=  1010 0100 =( 10 , 4 ): count 60
(X,Y)=  1011 1000 =( 11 , 8 ): count 56
(X,Y)=  1100 0001 =( 12 , 1 ): count 57
(X,Y)=  1101 0010 =( 13 , 2 ): count 59
(X,Y)=  1110 0100 =( 14 , 4 ): count 61
(X,Y)=  1111 1000 =( 15 , 8 ): count 70


In [11]:
qiskit.__version__

NameError: name 'qiskit' is not defined